# 1 ) Set up the environment:

In [1]:
!pip install transformers datasets torch bitsandbytes accelerate peft


In [7]:
import torch

print(f"Is CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device index: {torch.cuda.current_device()}")
else:
    print("CUDA is not available. You might be on a CPU-only runtime.")

Is CUDA available: True
Current device: NVIDIA A100-SXM4-40GB
Device count: 1
Current device index: 0


In [8]:
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"Is GPU available: {tf.test.is_gpu_available()}")
if tf.test.is_gpu_available():
    print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
else:
    print("No GPU found. You might be on a CPU-only runtime.")

TensorFlow version: 2.15.0
Is GPU available: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
!python --version
!cat /proc/cpuinfo | grep "model name" | uniq
!cat /proc/meminfo | grep MemTotal

Python 3.10.12
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
MemTotal:       87532028 kB


In [10]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current GPU:", torch.cuda.get_device_name(0))

Is CUDA available: True
Current GPU: NVIDIA A100-SXM4-40GB


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel

# 2) Load the model and tokenizer:

In [12]:
model_name = "bigscience/bloomz-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Model loaded on device: {model.device}")

Model loaded on device: cuda:0


# 3 ) Prepare the dataset:

In [13]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

def preprocess_function(examples):
    instruction = "You are a mental health assistant. Your job is to provide emotional support, actively listen, and offer practical suggestions for well-being. Respond empathically and do not give specific medical advice or diagnoses. Always make sure the user feels heard and supported. If the user mentions suicidal thoughts, encourage them to seek professional help immediately. Here's the conversation so far:\n\n"
    inputs = [f"{instruction}Human: {q}\nAI:" for q in examples["Context"]]
    targets = [f" {a}" for a in examples["Response"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

# Load the dataset
dataset = load_dataset("csv", data_files="dataset.csv")["train"]

# Create train-test split
train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)

# Convert back to Dataset objects
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2472 [00:00<?, ? examples/s]

Map:   0%|          | 0/275 [00:00<?, ? examples/s]

# 4 ) Configure LoRA:

In [14]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 5 ) Set up training arguments:

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,  # Enable mixed precision training for A100
    save_steps=100,
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    eval_steps=100,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 6 ) Train the model:

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)
trainer.train()

Step,Training Loss,Validation Loss
100,3.063500,3.047146
200,3.288900,3.009323
300,3.344600,2.985966
400,3.186500,2.973594


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=462, training_loss=3.498345832288007, metrics={'train_runtime': 436.264, 'train_samples_per_second': 16.999, 'train_steps_per_second': 1.059, 'total_flos': 2.751674602173235e+16, 'train_loss': 3.498345832288007, 'epoch': 2.9902912621359223})

# 7 ) Save the trained model to Google Drive:



In [17]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create a directory in Google Drive to save the model
save_directory = "/content/drive/MyDrive/mental_health_model"
os.makedirs(save_directory, exist_ok=True)

# Save the LoRA model
model.save_pretrained(f"{save_directory}/bloomz_1b7_lora")

# Save the tokenizer
tokenizer.save_pretrained(f"{save_directory}/bloomz_1b7_lora")

print(f"Model and tokenizer saved to Google Drive: {save_directory}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model and tokenizer saved to Google Drive: /content/drive/MyDrive/mental_health_model


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!zip -r /content/drive/MyDrive/mental_health_model.zip /content/drive/MyDrive/mental_health_model


updating: content/drive/MyDrive/mental_health_model/ (stored 0%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/ (stored 0%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/README.md (deflated 66%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/adapter_model.safetensors (deflated 8%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/adapter_config.json (deflated 50%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/tokenizer_config.json (deflated 73%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/special_tokens_map.json (deflated 79%)
updating: content/drive/MyDrive/mental_health_model/bloomz_1b7_lora/tokenizer.json (deflated 71%)


# 8 ) Push the model to Hugging Face Hub:

In [21]:
import os
from getpass import getpass

# Prompt for your Hugging Face token
hf_token = getpass("Enter your Hugging Face token: ")

# Set the token as an environment variable
os.environ["HUGGINGFACE_TOKEN"] = hf_token

Enter your Hugging Face token: ··········


In [22]:
import os
print("HUGGINGFACE_TOKEN is set:", "HUGGINGFACE_TOKEN" in os.environ)

HUGGINGFACE_TOKEN is set: True


In [24]:
#This block is only needed if after the training and save it for some reason the colab lost connection
'''
import os
print("Path exists:", os.path.exists("/content/drive/MyDrive/mental_health_model/bloomz_1b7_lora"))

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your saved model and tokenizer
model_path = "/content/drive/MyDrive/mental_health_model/bloomz_1b7_lora"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Model and tokenizer loaded successfully")
'''

Path exists: True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Model and tokenizer loaded successfully


In [23]:
from huggingface_hub import HfApi
import os

# Set your Hugging Face username and model name
hf_username = "darlingoscanoa"  # Replace with your Hugging Face username
model_name = "mental-health-assistant-bloomz-1b7"  # Choose a name for your model

# Use the token to log in
api = HfApi(token=os.environ["HUGGINGFACE_TOKEN"])

# Create a new model repository on the Hugging Face Hub
api.create_repo(repo_id=f"{hf_username}/{model_name}", exist_ok=True)

# Push the model to the Hugging Face Hub
model.push_to_hub(f"{hf_username}/{model_name}", token=os.environ["HUGGINGFACE_TOKEN"])
tokenizer.push_to_hub(f"{hf_username}/{model_name}", token=os.environ["HUGGINGFACE_TOKEN"])

print(f"Model pushed to https://huggingface.co/{hf_username}/{model_name}")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Model pushed to https://huggingface.co/darlingoscanoa/mental-health-assistant-bloomz-1b7


# 9 ) Test the published model:


In [24]:
hf_username = "darlingoscanoa"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model from the Hub
model_name = f"{hf_username}/mental-health-assistant-bloomz-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare a sample input
sample_text = "I've been feeling really down lately and I'm not sure what to do."

# Tokenize the input
inputs = tokenizer(sample_text, return_tensors="pt")

# Generate a response
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100, num_return_sequences=1, do_sample=True)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Sample Input:", sample_text)
print("Model Response:", response)

adapter_config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Sample Input: I've been feeling really down lately and I'm not sure what to do.
Model Response: I've been feeling really down lately and I'm not sure what to do. something it in something are to a is there is is do i i i be that are a in are be of to other one in what be in or who a on of up you


In [25]:
# Test with another input
sample_text2 = "How can I improve my mental health?"
inputs2 = tokenizer(sample_text2, return_tensors="pt")

with torch.no_grad():
    outputs2 = model.generate(**inputs2, max_length=100, num_return_sequences=1, do_sample=True)

response2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)
print("\nSample Input 2:", sample_text2)
print("Model Response 2:", response2)


Sample Input 2: How can I improve my mental health?
Model Response 2: How can I improve my mental health? not have in there have is i you of is to to be in your would you of be the in to of


In [26]:
# Test with another input
sample_text3 = "i want a secure relationship with someone that wants to be with me and who will actually put effort into it i seem to gravitate toward unavailable men and those that want intimacy and no relationship i let men dictate and control me because they accuse me of being controlling i let men emotionally abuse me and i am at their beck and call i am not comfortable being alone or doing anything by myself i feel i need the security of someone being around just to survive i know what i am doing wrong and i do it anyway just hoping things will change how do i stop this behavior and thought process"
inputs3 = tokenizer(sample_text3, return_tensors="pt")

with torch.no_grad():
    outputs3 = model.generate(**inputs3, max_length=500, num_return_sequences=1, do_sample=True)

response3 = tokenizer.decode(outputs3[0], skip_special_tokens=True)
print("\nSample Input 2:", sample_text3)
print("Model Response 2:", response3)







Sample Input 2: i want a secure relationship with someone that wants to be with me and who will actually put effort into it i seem to gravitate toward unavailable men and those that want intimacy and no relationship i let men dictate and control me because they accuse me of being controlling i let men emotionally abuse me and i am at their beck and call i am not comfortable being alone or doing anything by myself i feel i need the security of someone being around just to survive i know what i am doing wrong and i do it anyway just hoping things will change how do i stop this behavior and thought process
Model Response 2: i want a secure relationship with someone that wants to be with me and who will actually put effort into it i seem to gravitate toward unavailable men and those that want intimacy and no relationship i let men dictate and control me because they accuse me of being controlling i let men emotionally abuse me and i am at their beck and call i am not comfortable being alo

The results are not good

# 10 ) Evaluación del Rendimiento del Modelo con Métricas ROUGE

In [28]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=643bf0cc402044c49dd98718f3eec07eb54cace2669f13307c88f0410e880eaf
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
from rouge_score import rouge_scorer
import numpy as np
from transformers import pipeline
import os

def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_f1 = []
    rouge2_f1 = []
    rougeL_f1 = []

    for pred, ref in zip(predictions, references):
        scores = scorer.score(pred, ref)
        rouge1_f1.append(scores['rouge1'].fmeasure)
        rouge2_f1.append(scores['rouge2'].fmeasure)
        rougeL_f1.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': np.mean(rouge1_f1),
        'rouge2': np.mean(rouge2_f1),
        'rougeL': np.mean(rougeL_f1)
    }

In [31]:
#Loading the fine tuned model
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set your Hugging Face username and model name
hf_username = "darlingoscanoa"  # Replace if different
model_name = "mental-health-assistant-bloomz-1b7"
model_id = f"{hf_username}/{model_name}"

try:
    # Try to load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"])
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"])

    print(f"Model and tokenizer loaded successfully from {model_id}")

    # Create the generator pipeline
    from transformers import pipeline
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

except Exception as e:
    print(f"Error loading model from Hugging Face: {e}")

    # If loading from Hugging Face fails, try loading from local path
    local_model_path = "/content/drive/MyDrive/mental_health_model/bloomz_1b7_lora"
    try:
        model = AutoModelForCausalLM.from_pretrained(local_model_path)
        tokenizer = AutoTokenizer.from_pretrained(local_model_path)
        print(f"Model and tokenizer loaded successfully from local path: {local_model_path}")

        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    except Exception as e:
        print(f"Error loading model from local path: {e}")
        print("Please check the model path and ensure the model files are present.")

Model and tokenizer loaded successfully from darlingoscanoa/mental-health-assistant-bloomz-1b7


In [33]:
#Load the model and tokenizer
hf_username = "darlingoscanoa"  # Replace if different
model_name = "mental-health-assistant-bloomz-1b7"
model_id = f"{hf_username}/{model_name}"

model = AutoModelForCausalLM.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"])
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"])

print(f"Model and tokenizer loaded successfully from {model_id}")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


Model and tokenizer loaded successfully from darlingoscanoa/mental-health-assistant-bloomz-1b7


In [36]:
import pandas as pd
#Load and prepare the dataset
#dataset_path = "/content/drive/MyDrive/dataset.csv"
dataset_path = "./dataset.csv"

df = pd.read_csv(dataset_path)

# Randomly select 3 examples
sample_size = 3
sample_df = df.sample(n=sample_size, random_state=42)

In [45]:
# Step 10.5: Generate responses and prepare for evaluation
test_prompts = []
predictions = []
references = []

for _, row in sample_df.iterrows():
    context = row['Context']
    reference = row['Response']

    prompt = f"Human: {context}\nAI:"
    test_prompts.append(prompt)

    # Generate response with explicit truncation and a reasonable max_length
    response = generator(prompt, max_length=512, num_return_sequences=1, truncation=True, do_sample=True, temperature=0.7)

    # Extract the generated text after "AI:"
    generated_text = response[0]['generated_text']
    ai_response = generated_text.split("AI:")[-1].strip()

    # If the model didn't generate anything after "AI:", use the whole generated text
    prediction = ai_response if ai_response else generated_text

    predictions.append(prediction)
    references.append(reference)


In [46]:
# Step 10.6: Define ROUGE calculation function
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_f1 = []
    rouge2_f1 = []
    rougeL_f1 = []

    for pred, ref in zip(predictions, references):
        scores = scorer.score(pred, ref)
        rouge1_f1.append(scores['rouge1'].fmeasure)
        rouge2_f1.append(scores['rouge2'].fmeasure)
        rougeL_f1.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': np.mean(rouge1_f1),
        'rouge2': np.mean(rouge2_f1),
        'rougeL': np.mean(rougeL_f1)
    }

In [47]:
# Step 10.7: Calculate and display ROUGE scores
rouge_scores = calculate_rouge(predictions, references)

print("ROUGE Scores:")
for metric, score in rouge_scores.items():
    print(f"{metric}: {score:.4f}")


ROUGE Scores:
rouge1: 0.1195
rouge2: 0.0000
rougeL: 0.0760


In [48]:
# Step 10.8: Display predictions and references
print("\nPredictions vs References:")
for i, (prompt, pred, ref) in enumerate(zip(test_prompts, predictions, references)):
    print(f"\nExample {i+1}:")
    print(f"Context: {prompt.split('Human: ')[1].split('AI:')[0].strip()}")
    print(f"AI Prediction: {pred}")
    print(f"Reference: {ref}")
    print("-" * 50)


Predictions vs References:

Example 1:
Context: i keep having these random thoughts that i do not want things like you are not worth anything i know they are my own thoughts but it feels like someone else is saying it what is wrong with me and how can i stop having these thoughts
AI Prediction: your or would if a you a for to i a you and a that that and and and of on
Reference: first off there is nothing wrong with you in fact having random thoughts we feel we cannot control is actually quite common and normal sometimes our thoughts think they are protecting us if we think we are not worth anything it will not hurt when we get rejected but they are actually doing quite the opposite they are keeping us stuck and creating a selfprophization if i think i am not worth anything i do not have to try and i will keep proving to myself that i am not worth anything there are many techniques to work through and start to change our negative selftalk there are actually many selfhelp workbooks that

In [49]:
# Step 10.9: Additional check to display raw model output
print("\nRaw Model Outputs:")
for i, prompt in enumerate(test_prompts):
    raw_output = generator(prompt, max_length=512, num_return_sequences=1, truncation=True, do_sample=True, temperature=0.7)
    print(f"\nExample {i+1} Raw Output:")
    print(raw_output[0]['generated_text'])
    print("-" * 50)


Raw Model Outputs:

Example 1 Raw Output:
Human: i keep having these random thoughts that i do not want things like you are not worth anything i know they are my own thoughts but it feels like someone else is saying it what is wrong with me and how can i stop having these thoughts
AI: and the a so you in long to
--------------------------------------------------

Example 2 Raw Output:
Human: i have a friend that who i used to be in a relationship with it was brief and turned into us being just good friends i spent the weekend with him and it upset my boyfriend was i wrong
AI: it not just a as if and to no not to so your to and of
--------------------------------------------------

Example 3 Raw Output:
Human: i think i have depression anxiety bipolar disorder and i have anger issues i told my mom over and over again she told me she would get me a therapist but she never did i just found out that my mom cannot afford a therapist for me
AI: the about be in the a and long help and is in 